In [ ]:
with open("../input/airport-reviews/airport_reviews.csv", encoding="utf-8") as fp:
    reviews_text = fp.read()

In [ ]:
len(reviews_text)

reviews_text = reviews_text[:3000]

In [ ]:
reviews_text

In [ ]:
chars_list = sorted(list(set(reviews_text)))
char_to_index_dict = {
    character: chars_list.index(character) for character in chars_list
}
                    

In [ ]:
print(char_to_index_dict)

### Lets construct an RNN to learn and predict the sequence of characters

In [ ]:
import keras
from keras import layers

max_length = 40
rnn = keras.models.Sequential()

rnn.add(layers.LSTM(1024, input_shape=(max_length, len(chars_list)), return_sequences=True))
rnn.add((layers.LSTM(1024, input_shape=(max_length, len(chars_list)))))
rnn.add(layers.Dense(len(chars_list), activation="softmax"))

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-6, nesterov=True)
rnn.compile(loss="categorical_crossentropy", optimizer=optimizer)

In [ ]:
import numpy as np

def text_to_vector(input_txt, max_length):
    sentences = []
    next_characters = []
    for i in range(0, len(input_txt) - max_length):
        sentences.append(input_txt[i:i+ max_length])
        next_characters.append(input_txt[i + max_length])
    
    X = np.zeros((len(sentences), max_length, len(chars_list)))
    y = np.zeros((len(sentences), len(chars_list)))
    
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            X[i, t, char_to_index_dict[char]] = 1
            y[i, char_to_index_dict[next_characters[i]]] =1
    return [X, y]

In [ ]:
X, y = text_to_vector(reviews_text, max_length)
rnn.fit(X, y, batch_size=256, epochs=1)

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
import random

maxlen = 40
char_indices = dict((c, i) for i, c in enumerate(chars_list))
indices_char = dict((i, c) for i, c in enumerate(chars_list))

def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(reviews_text) - maxlen - 1)
    generated = ''
    sentence = reviews_text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars_list)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = rnn.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [ ]:
generate_text(500, 0.2)

something doesnt feel right, hehe